In [39]:
import sqlite3
import yaml
import os, random
import pandas as pd
import psycopg2
import yaml
import gender_guesser.detector as gender_detector
import urllib.request
import shutil

In [40]:
path = '~/Desktop/database_info.yml'
LFW_DIR = 'lfw'
DEST_DIR = 'upload_images'
d = gender_detector.Detector()
homo_prob = 0.017
bi_prob = 1 - 0.01
url_path = 'https://perfect-match-pictures.s3.amazonaws.com/lfw/'
not_exist_users = []

In [41]:
def url_is_alive(url):
    """
    Checks that a given URL is reachable.
    :param url: A URL
    :rtype: bool
    """
    request = urllib.request.Request(url)
    request.get_method = lambda: 'HEAD'

    try:
        urllib.request.urlopen(request)
        return True
    except urllib.request.HTTPError:
        return False

In [42]:
# Obtain the configuration parameters
with open(os.path.expanduser(path)) as file:
    data = yaml.load(file, Loader=yaml.FullLoader)
    database_core = data['perfect_match']

POSTGRES_HOST = database_core['host']
POSTGRES_PORT = database_core['port']
POSTGRES_DATABASE = database_core['database']
POSTGRES_USERNAME = database_core['username']
POSTGRES_PASSWORD = database_core['password']

# Connecting to the PostgresSQL database
conn = psycopg2.connect(
    host = POSTGRES_HOST,
    port = POSTGRES_PORT,
    database = POSTGRES_DATABASE,
    user = POSTGRES_USERNAME,
    password = POSTGRES_PASSWORD,
)
# Creating a cursor
cur = conn.cursor()

# Function receives a query and return a pandas table
def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, conn)
    return table

In [43]:
insert_user_query = """
    INSERT INTO users (username, gender, preference)
    VALUES (%s,%s,%s)
"""
def get_user_id(username):
    return """
        SELECT id
        FROM users
        WHERE username = '{}'
    """.format(username)

insert_images_query = """
    INSERT INTO images (user_id, url)
    VALUES (%s,%s)
"""

In [44]:
total = 0
gender_male = 0
gender_female = 0
gender_other = 0
preference_male = 0
preference_female = 0
preference_all = 0
count = 0

for directory in os.listdir(LFW_DIR):
    if directory == '.DS_Store':
        continue

    if count < 5319:
        count+=1
        continue
    
    urls = []
    username = directory.replace('_', ' ')
    
    for file in os.listdir(os.path.join(LFW_DIR, directory)):
        if file == '.DS_Store':
            continue
        
        url = url_path + directory + '/' + file
        if url_is_alive(url) is True:
            urls.append(url)
        
    # User has no images
    if len(urls) == 0:
        print('User', username, 'does not exist')
        not_exist.append(username)
        shutil.copytree(os.path.join(LFW_DIR, directory), os.path.join(DEST_DIR, directory))
        continue

    
    gender_category = d.get_gender(username.split()[0])
    
    # Get preference and define gender
    prob = random.random()
    if gender_category == 'female' or gender_category == 'mostly_female':
        gender = 'female'
        gender_female+=1
        if prob < homo_prob:
            preference = 'female'
            preference_female+=1
        elif prob > bi_prob:
            preference = 'all'
            preference_all+=1
        else:
            preference = 'male'
            preference_male+=1
    elif gender_category == 'male' or gender_category == 'mostly_male':
        gender = 'male'
        gender_male+=1
        if prob < homo_prob:
            preference = 'male'
            preference_male+=1
        elif prob > bi_prob:
            preference = 'all'
            preference_all+=1
        else:
            preference = 'female'
            preference_female+=1
    else:
        gender = 'other'
        gender_other+=1
        if prob < 0.4:
            preference = 'female'
            preference_female+=1
        elif prob > 0.6:
            preference = 'male'
            preference_male+=1
        else:
            preference = 'all'
            preference_all+=1
    
    print(username, gender, preference, len(urls))
    
    # Insert in users
    records_to_insert_users = (username, gender, preference)
    cur.execute(insert_user_query, records_to_insert_users)
    conn.commit()
    
    # Get user_id
    user_id_query = get_user_id(username)
    data = create_pandas_table(user_id_query)
    user_id = data.id[0].item()
    
    # Insert images
    for url in urls:
        record_to_insert_images = (user_id, url)
        cur.execute(insert_images_query, record_to_insert_images)
        conn.commit()
    
    count+=1
    total+=1

Michael Lechner male female 1


/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Matthew Ouimet male female 1


/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Edmund Hillary male female 3


...

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Roger King male female 1


/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Michel Minard male female 1


/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [26]:
print(
    'total',
    total,
    'not_exist',
    len(not_exist_users),
    'gender_male',
    gender_male,
    'gender_female',
    gender_female,
    'gender_other',
    gender_other,
    'preference_female',
    preference_female,
    'preference_male',
    preference_male,
    'preference_all',
    preference_all,
)

total 962 not_exist 0 gender_male 623 gender_female 247 gender_other 92 preference_female 655 preference_male 278 preference_all 29


In [28]:
# Closing the cursor and connection to server
cur.close()
conn.close()

In [38]:
count = 0
for directory in os.listdir(LFW_DIR):
    if directory == '.DS_Store':
        continue
    
    if count < 5318:
        count+=1
        continue
    
    username = directory.replace('_', ' ')
    if username == 'Kirk Doerger':
        print(count)

    print(username)
    count+=1

5318
Kirk Doerger
Michael Lechner
Matthew Ouimet
Edmund Hillary
Robbie Williams
Noer Muis
Michel Kratochvil
Rachel Hunter
Teri Garr
Taku Yamasaki
Chris Thomas
Mike Easley
Bob Iger
Steve Peace
Mickey Sherman
Princess Stephanie
Daniel Chin
Steve Rush
Janet Horvath
Joan Dangerfield
Silvan Shalom
Gregorio Rosal
Lucio Cecchinello
Theo Angelopoulos
Atsushi Sato
Mike Gable
Jerry Regier
Ivan Helguera
Turner Stevenson
Normand Legault
Evan Rachel Wood
Roy Romanow
Bob Beauprez
Gregory Hines
Lily Tomlin
Seth Gorney
Federico Trillo
Maureen Fanning
Jawad Boulus
Melchor Cob Castro
Suzie McConnell Serio
Cheryl Tiegs
Lee Yuan-tseh
Vyacheslav Fetisov
Nathalie Baye
Chuck Eidson
Ray Wasden
Mark Podlesny
Luc Montagnier
Ali Khamenei
Basdeo Panday
Andy Benes
Galen Rowell
Gregg Berhalter
Stefanie De Roux
Paul Farley
Kevin Gil
Kevin Borseth
Richard Chamberlain
Benedita da Silva
Tom Moss
Wim Duisenberg
Chang Dae-whan
Michael J Fox
Oleg Romantsev
Terunobu Maeda
Ron Zook
Bruce Willis
Jason Kapono
Bertie Ahern
Alm